In [1]:
import pandas as pd
import os
import sys
import requests

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
cwd = os.getcwd()

education_folder_path = os.path.dirname(cwd)

sys.path.append(education_folder_path)

In [3]:
from global_functions import remove_spaces_in_columns, up_load_df

In [4]:
#בתי ספר ממשרד החינוך בכל הארץ
Gschool=up_load_df(r'{}\background_files\ministry_of_education\מוסדות'.format(education_folder_path),'schools_2023')
Gschool = remove_spaces_in_columns(Gschool)

In [5]:
#יישובים בשטח צתאל שם וקוד יישוב
JTMT_setls=up_load_df(r'{}\background_files'.format(education_folder_path),'210615_מקוצר_רשימת_יישובים_באחריות_צתאל')
JTMT_setls = remove_spaces_in_columns(JTMT_setls)

In [6]:
Gschool=Gschool[Gschool['יישוב'].isin(JTMT_setls['שם_יישוב'])]

In [7]:
Gschool=Gschool[Gschool['יישוב'] != 'ירושלים']
Gschool=Gschool[Gschool['יישוב'] != 'בית שמש']

In [8]:
#קאורדינטות של מוסדות חינוך בכל הארץ ממשרד החינוך
moe_mosdot_coordinates_2022=up_load_df(r'{}\background_files\ministry_of_education\מוסדות'.format(education_folder_path),'moe_mosdot_coordinates_2022')
moe_mosdot_coordinates_2022=remove_spaces_in_columns(moe_mosdot_coordinates_2022)

In [9]:
moe_mosdot_coordinates_2022.rename(columns={'SEMEL_MOSAD': 'סמל_מוסד'}, inplace=True)
moe_mosdot_coordinates_2022.rename(columns={'SHEM_MOSAD': 'שם מוסד'}, inplace=True)
moe_mosdot_coordinates_2022.rename(columns={'ITM_X': 'coordinate_x'}, inplace=True)
moe_mosdot_coordinates_2022.rename(columns={'ITM_Y': 'coordinate_y'}, inplace=True)

moe_mosdot_coordinates_2022 = moe_mosdot_coordinates_2022.drop(columns=['UTM_X', 'UTM_Y', 'RAMAT_DIYUK_MIKUM'])

In [10]:
# מיזוג הטבלאות לפי העמודה "סמל_מוסד"
Gschool = pd.merge(Gschool, moe_mosdot_coordinates_2022[['סמל_מוסד', 'coordinate_x', 'coordinate_y']],
                     on='סמל_מוסד', how='left')

In [11]:
Gschool.head()

,סמל_מוסד,שם_מוסד,יישוב,כתובת,"כתובת_חט""ב",טלפון,"דוא""ל_מזכירות",סוג_חינוך,מעמד_משפטי,סוג_פיקוח,מגזר,יחידת_דיווח,שכבה,מספר_תלמידים,coordinate_x,coordinate_y
0,110098,השחר,טל שחר,טל שחר,NaN,08-9349345,NaN,רגיל,רשמי,ממלכתי,יהודי,ירושלים,א - ו,420.00,190867.00,634787.00
1,110726,מתתיהו,צלפון,צלפון 5,NaN,02-9911647,sigalyeh1@hinuchm.k12.il,רגיל,רשמי,ממלכתי דתי,יהודי,ירושלים,א - ו,349.00,194187.00,634711.00
2,110734,הר טוב,צרעה,צרעה 28,NaN,02-9908900,NaN,רגיל,רשמי,ממלכתי,יהודי,ירושלים,א - ו,513.00,197528.00,630138.00
3,111096,עין הרים,"עין כרם-בי""ס חקלאי","עין כרם-בי""ס חקלא 5",NaN,02-6431318,yaelelk1@hinuchm.k12.il,רגיל,רשמי,ממלכתי,יהודי,ירושלים,א - ו,545.00,214636.00,631692.00
4,111195,חברון בנות,קרית ארבע,נחל אשכול 407,NaN,02-9961050,zipporahapo@hinuchm.k12.il,רגיל,רשמי,ממלכתי דתי,יהודי,ירושלים,א - ו,293.00,211257.00,604302.00


In [12]:
Gschool=Gschool.drop(columns=['כתובת_חט"ב', "טלפון", 'דוא"ל_מזכירות', "מעמד_משפטי"])

In [13]:
Gschool=Gschool[Gschool['coordinate_x'] != 0]

In [14]:
# שמירת רק השורות שבהן המילה "גן" מופיעה בעמודה "שם_מוסד"
df = Gschool[Gschool['שכבה'].str.contains('גן', na=False)]
df

,סמל_מוסד,שם_מוסד,יישוב,כתובת,סוג_חינוך,סוג_פיקוח,מגזר,יחידת_דיווח,שכבה,מספר_תלמידים,coordinate_x,coordinate_y
27,112367,מגילות,קליה,קליה,רגיל,ממלכתי,יהודי,ירושלים,גן - ו,267.00,244247.00,628899.00
62,115527,עידנים,מודיעין-מכבים-רעות,תלתן 2,רגיל,ממלכתי,יהודי,ירושלים,גן - ו,273.00,200811.00,645977.00
63,115550,דורות,מודיעין-מכבים-רעות,גינת החולה 12,רגיל,ממלכתי,יהודי,ירושלים,גן - ו,292.00,202260.00,644699.00
65,115642,היובל,מודיעין-מכבים-רעות,גינת איילון 34,רגיל,ממלכתי,יהודי,ירושלים,גן - ו,277.00,201462.00,645735.00
66,115675,צלילים,מודיעין-מכבים-רעות,תרשיש 73,מיוחד,ממלכתי,יהודי,ירושלים,גן - יב,9.00,NaN,NaN
68,116012,אופק,מודיעין-מכבים-רעות,גינת דותן 11,רגיל,ממלכתי,יהודי,ירושלים,גן - ו,250.00,202412.00,645100.00
71,120014,נוף הרים,איתנים,איתנים,מיוחד,ממלכתי,יהודי,ירושלים,גן - יב,NaN,209209.00,631804.00
86,131300,אסיף,מודיעין-מכבים-רעות,עמק בית שאן 4,רגיל,ממלכתי,יהודי,ירושלים,גן - ו,450.00,200619.00,646368.00
87,131318,"ע""ש אילן רמון",מודיעין-מכבים-רעות,"רא""ל חיים ברלב 45",רגיל,ממלכתי,יהודי,ירושלים,גן - ו,314.00,199987.00,646279.00
155,172189,קשת,מודיעין-מכבים-רעות,חטיבת הצנחנים 1,רגיל,ממלכתי,יהודי,ירושלים,גן - ו,282.00,200171.00,645736.00


In [15]:
Gschool=Gschool[~Gschool['שכבה'].str.contains('גן', na=False)]

In [16]:
GschoolXYNaN = Gschool[Gschool['coordinate_x'].isna()]
Gschool = Gschool[~Gschool['coordinate_x'].isna()]

In [17]:
# הוסף עמודות לקואורדינטות
GschoolXYNaN['coordinate_x'] = None
GschoolXYNaN['coordinate_y'] = None

# פונקציה לשליחת בקשה
def get_coordinates(address, settlement):
    # קידוד הכתובת והיישוב
    query_address = f"{address}, {settlement}"
    # encoded_address = urllib.parse.quote(query_address)
    
    # יצירת URL
    url = f"https://es.govmap.gov.il/TldSearch/api/DetailsByQuery?query={query_address}&lyrs=276267023&gid=govmap"
    # print(url)
    # ביצוע הבקשה
    response = requests.get(url)
    if response.status_code == 200:
        # print(response.json())
        return response.json()  # מחזיר את התגובה כ-json
    else:
        print(f"Error fetching data for {query_address}: {response.status_code}")
        return None

# עובר על כל שורה בטבלה ומבצע את הבקשה
for index, row in GschoolXYNaN.iterrows():
    coordinates = get_coordinates(row['כתובת'], row['יישוב'])
    if coordinates:
        if coordinates['data'] != {}:
            data=coordinates['data']
            first_value = next(iter(data.values()))
            # print(first_value[0])
            values = iter(first_value[0].values())
            x = next(values)
            y = next(values)
            GschoolXYNaN.loc[index, 'coordinate_x'] = x
            GschoolXYNaN.loc[index, 'coordinate_y'] = y

GschoolXYNotNaN = GschoolXYNaN[~GschoolXYNaN['coordinate_x'].isna()]

In [18]:
Gschool.head()

,סמל_מוסד,שם_מוסד,יישוב,כתובת,סוג_חינוך,סוג_פיקוח,מגזר,יחידת_דיווח,שכבה,מספר_תלמידים,coordinate_x,coordinate_y
0,110098,השחר,טל שחר,טל שחר,רגיל,ממלכתי,יהודי,ירושלים,א - ו,420.00,190867.00,634787.00
1,110726,מתתיהו,צלפון,צלפון 5,רגיל,ממלכתי דתי,יהודי,ירושלים,א - ו,349.00,194187.00,634711.00
2,110734,הר טוב,צרעה,צרעה 28,רגיל,ממלכתי,יהודי,ירושלים,א - ו,513.00,197528.00,630138.00
3,111096,עין הרים,"עין כרם-בי""ס חקלאי","עין כרם-בי""ס חקלא 5",רגיל,ממלכתי,יהודי,ירושלים,א - ו,545.00,214636.00,631692.00
4,111195,חברון בנות,קרית ארבע,נחל אשכול 407,רגיל,ממלכתי דתי,יהודי,ירושלים,א - ו,293.00,211257.00,604302.00


In [19]:
GschoolXYNotNaN.head()

,סמל_מוסד,שם_מוסד,יישוב,כתובת,סוג_חינוך,סוג_פיקוח,מגזר,יחידת_דיווח,שכבה,מספר_תלמידים,coordinate_x,coordinate_y
64,115600,ארץ הצבר,מודיעין-מכבים-רעות,גולדה מאיר 28,רגיל,ממלכתי,יהודי,ירושלים,א - ו,154.00,645483.44,198960.81
133,160077,דרכי שלום,גבעת יערים,גבעת יערים,רגיל,חרדי,יהודי,מוכרים,ט - יב,NaN,632777.93,208403.97
378,484204,שש שנתי תורני לבנים נופים,נופים,נופים,רגיל,ממלכתי דתי,יהודי,מרכז,ז - יב,0.00,673564.46,209854.19
460,658880,ישיבת היכל התורה,ביתר עילית,"אדמור""י ויז'ניץ 25",רגיל,חרדי,יהודי,מוכרים,ט - יא,NaN,622517.92,209453.34
461,658898,ישיבה תיכונית צביה,משואה,משואה,רגיל,ממלכתי דתי,יהודי,ירושלים,ז - יב,87.00,668863.69,246471.06


In [20]:
Gschool=pd.concat([Gschool, GschoolXYNotNaN])
Gschool.to_excel('Gschool.xlsx', index=False)
Gschool.head()

,סמל_מוסד,שם_מוסד,יישוב,כתובת,סוג_חינוך,סוג_פיקוח,מגזר,יחידת_דיווח,שכבה,מספר_תלמידים,coordinate_x,coordinate_y
0,110098,השחר,טל שחר,טל שחר,רגיל,ממלכתי,יהודי,ירושלים,א - ו,420.00,190867.00,634787.00
1,110726,מתתיהו,צלפון,צלפון 5,רגיל,ממלכתי דתי,יהודי,ירושלים,א - ו,349.00,194187.00,634711.00
2,110734,הר טוב,צרעה,צרעה 28,רגיל,ממלכתי,יהודי,ירושלים,א - ו,513.00,197528.00,630138.00
3,111096,עין הרים,"עין כרם-בי""ס חקלאי","עין כרם-בי""ס חקלא 5",רגיל,ממלכתי,יהודי,ירושלים,א - ו,545.00,214636.00,631692.00
4,111195,חברון בנות,קרית ארבע,נחל אשכול 407,רגיל,ממלכתי דתי,יהודי,ירושלים,א - ו,293.00,211257.00,604302.00


In [21]:
GschoolXYNaN = GschoolXYNaN[GschoolXYNaN['coordinate_x'].isna()]
GschoolXYNaN

,סמל_מוסד,שם_מוסד,יישוב,כתובת,סוג_חינוך,סוג_פיקוח,מגזר,יחידת_דיווח,שכבה,מספר_תלמידים,coordinate_x,coordinate_y
437,632323,תורת חיים ביתר,ביתר עילית,הרב בנדר 10,רגיל,חרדי,יהודי,מעיין החינוך התורני,א - ד,80.00,None,None
441,637231,"בית ספר חב""ד יצהר",יצהר,יצהר 770,רגיל,חרדי,יהודי,מוכרים,א - ו,NaN,None,None
455,655233,שירת ציון,כוכב יעקב,אהבת ישראל 57,רגיל,חרדי,יהודי,מוכרים,ט - י,28.00,None,None
464,659359,חזון איש,מודיעין עילית,שד אביי ורבא 16,רגיל,חרדי,יהודי,מוכרים,א - ד,NaN,None,None
467,659888,הר עציון לצעירים,אלון שבות,הישיבה 1,רגיל,ממלכתי דתי,יהודי,ירושלים,ט - יב,NaN,None,None
475,673426,נחלת צבי,ביתר עילית,אש קודש 22,מיוחד,חרדי,יהודי,"מוכרים - חנ""מ וחט""ב",א - יב,NaN,None,None
476,673483,מאור התורה,עמנואל,הגאון מוילנה 27,מיוחד,חרדי,יהודי,מעיין החינוך התורני,א - יב,NaN,None,None
479,675108,תלמוד תורה גבעות עול,גבעת זאב,האיילות 50,רגיל,חרדי,יהודי,מוכרים,א - ח,NaN,None,None
480,675348,מכינה לישיבה ליובאוו,ביתר עילית,הרבי מנדבורנה 8,מיוחד,חרדי,יהודי,"מוכרים - חנ""מ וחט""ב",ז - יב,NaN,None,None


In [22]:
import re

# פונקציה שמסירה מספרים מכתובת ומשאירה רק את הטקסט
def remove_numbers(address):
    return re.sub(r'\d+', '', address).strip()

# החלת הפונקציה על עמודת "כתובת"
GschoolXYNaN['כתובת'] = GschoolXYNaN['כתובת'].apply(remove_numbers)

# הצגת הטבלה המעודכנת
GschoolXYNaN

,סמל_מוסד,שם_מוסד,יישוב,כתובת,סוג_חינוך,סוג_פיקוח,מגזר,יחידת_דיווח,שכבה,מספר_תלמידים,coordinate_x,coordinate_y
437,632323,תורת חיים ביתר,ביתר עילית,הרב בנדר,רגיל,חרדי,יהודי,מעיין החינוך התורני,א - ד,80.00,None,None
441,637231,"בית ספר חב""ד יצהר",יצהר,יצהר,רגיל,חרדי,יהודי,מוכרים,א - ו,NaN,None,None
455,655233,שירת ציון,כוכב יעקב,אהבת ישראל,רגיל,חרדי,יהודי,מוכרים,ט - י,28.00,None,None
464,659359,חזון איש,מודיעין עילית,שד אביי ורבא,רגיל,חרדי,יהודי,מוכרים,א - ד,NaN,None,None
467,659888,הר עציון לצעירים,אלון שבות,הישיבה,רגיל,ממלכתי דתי,יהודי,ירושלים,ט - יב,NaN,None,None
475,673426,נחלת צבי,ביתר עילית,אש קודש,מיוחד,חרדי,יהודי,"מוכרים - חנ""מ וחט""ב",א - יב,NaN,None,None
476,673483,מאור התורה,עמנואל,הגאון מוילנה,מיוחד,חרדי,יהודי,מעיין החינוך התורני,א - יב,NaN,None,None
479,675108,תלמוד תורה גבעות עול,גבעת זאב,האיילות,רגיל,חרדי,יהודי,מוכרים,א - ח,NaN,None,None
480,675348,מכינה לישיבה ליובאוו,ביתר עילית,הרבי מנדבורנה,מיוחד,חרדי,יהודי,"מוכרים - חנ""מ וחט""ב",ז - יב,NaN,None,None


In [23]:
# הוסף עמודות לקואורדינטות
GschoolXYNaN['coordinate_x'] = None
GschoolXYNaN['coordinate_y'] = None

# פונקציה לשליחת בקשה
def get_coordinates(address, settlement):
    # קידוד הכתובת והיישוב
    query_address = f"{address}, {settlement}"
    # encoded_address = urllib.parse.quote(query_address)
    
    # יצירת URL
    url = f"https://es.govmap.gov.il/TldSearch/api/DetailsByQuery?query={query_address}&lyrs=276267023&gid=govmap"
    # print(url)
    # ביצוע הבקשה
    response = requests.get(url)
    if response.status_code == 200:
        # print(response.json())
        return response.json()  # מחזיר את התגובה כ-json
    else:
        print(f"Error fetching data for {query_address}: {response.status_code}")
        return None

# עובר על כל שורה בטבלה ומבצע את הבקשה
for index, row in GschoolXYNaN.iterrows():
    coordinates = get_coordinates(row['כתובת'], row['יישוב'])
    if coordinates:
        if coordinates['data'] != {}:
            data=coordinates['data']
            first_value = next(iter(data.values()))
            # print(first_value[0])
            values = iter(first_value[0].values())
            x = next(values)
            y = next(values)
            GschoolXYNaN.loc[index, 'coordinate_x'] = x
            GschoolXYNaN.loc[index, 'coordinate_y'] = y

GschoolXYNotNaN = GschoolXYNaN[~GschoolXYNaN['coordinate_x'].isna()]

In [24]:
GschoolXYNotNaN

,סמל_מוסד,שם_מוסד,יישוב,כתובת,סוג_חינוך,סוג_פיקוח,מגזר,יחידת_דיווח,שכבה,מספר_תלמידים,coordinate_x,coordinate_y
437,632323,תורת חיים ביתר,ביתר עילית,הרב בנדר,רגיל,חרדי,יהודי,מעיין החינוך התורני,א - ד,80.00,623280.39,211355.72
441,637231,"בית ספר חב""ד יצהר",יצהר,יצהר,רגיל,חרדי,יהודי,מוכרים,א - ו,NaN,675004.54,222546.48
455,655233,שירת ציון,כוכב יעקב,אהבת ישראל,רגיל,חרדי,יהודי,מוכרים,ט - י,28.00,643031.58,222685.79
464,659359,חזון איש,מודיעין עילית,שד אביי ורבא,רגיל,חרדי,יהודי,מוכרים,א - ד,NaN,649569.48,204576.84
467,659888,הר עציון לצעירים,אלון שבות,הישיבה,רגיל,ממלכתי דתי,יהודי,ירושלים,ט - יב,NaN,618474.64,211822.53
475,673426,נחלת צבי,ביתר עילית,אש קודש,מיוחד,חרדי,יהודי,"מוכרים - חנ""מ וחט""ב",א - יב,NaN,622258.06,211112.22
476,673483,מאור התורה,עמנואל,הגאון מוילנה,מיוחד,חרדי,יהודי,מעיין החינוך התורני,א - יב,NaN,674074.58,213008.12
479,675108,תלמוד תורה גבעות עול,גבעת זאב,האיילות,רגיל,חרדי,יהודי,מוכרים,א - ח,NaN,641182.93,214520.15
480,675348,מכינה לישיבה ליובאוו,ביתר עילית,הרבי מנדבורנה,מיוחד,חרדי,יהודי,"מוכרים - חנ""מ וחט""ב",ז - יב,NaN,622209.86,210800.25


In [25]:
Gschool=pd.concat([Gschool, GschoolXYNotNaN])
Gschool.to_excel('Gschool_final.xlsx', index=False)
Gschool.head()

,סמל_מוסד,שם_מוסד,יישוב,כתובת,סוג_חינוך,סוג_פיקוח,מגזר,יחידת_דיווח,שכבה,מספר_תלמידים,coordinate_x,coordinate_y
0,110098,השחר,טל שחר,טל שחר,רגיל,ממלכתי,יהודי,ירושלים,א - ו,420.00,190867.00,634787.00
1,110726,מתתיהו,צלפון,צלפון 5,רגיל,ממלכתי דתי,יהודי,ירושלים,א - ו,349.00,194187.00,634711.00
2,110734,הר טוב,צרעה,צרעה 28,רגיל,ממלכתי,יהודי,ירושלים,א - ו,513.00,197528.00,630138.00
3,111096,עין הרים,"עין כרם-בי""ס חקלאי","עין כרם-בי""ס חקלא 5",רגיל,ממלכתי,יהודי,ירושלים,א - ו,545.00,214636.00,631692.00
4,111195,חברון בנות,קרית ארבע,נחל אשכול 407,רגיל,ממלכתי דתי,יהודי,ירושלים,א - ו,293.00,211257.00,604302.00


In [26]:
is_unique = Gschool['סמל_מוסד'].is_unique
is_unique

True